In [1]:
#Importamos librerias:
import numpy as np
import pandas as pd
pd.options.mode.copy_on_write = True
import matplotlib.pyplot as plt
import seaborn as sns
import math
# from scipy.stats import 

In [1]:
2+2+6

10

In [6]:
#Cargamos el dataframe del titanic:
df=pd.read_csv("./data/titanic.csv")

In [7]:
df.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [8]:
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

#### Función describe

In [9]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [26]:
def describe_df(df):
    DATA_TYPE=df.dtypes
    MISSINGS=(df.isna().sum()/len(df)*100).sort_values(ascending=False)
    UNIQUE_VALUES=df.nunique()
    CARDIN=UNIQUE_VALUES/len(df)*100
    describe_df=pd.DataFrame([DATA_TYPE, MISSINGS, UNIQUE_VALUES, CARDIN])
    parametros=["DATA_TYPE", "MISSINGS (%)", "UNIQUE_VALUES", "CARDIN (%)"]
    describe_df.insert(0, "COL_N",parametros)
    return describe_df

In [27]:
describe_df(df)

,COL_N,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,DATA_TYPE,int64,int64,object,float64,int64,int64,float64,object,object,object,bool,object,object,object,bool
1,MISSINGS (%),0.0,0.0,0.0,19.86532,0.0,0.0,0.0,0.224467,0.0,0.0,0.0,77.216611,0.224467,0.0,0.0
2,UNIQUE_VALUES,2,3,2,88,7,7,248,3,3,3,2,7,3,2,2
3,CARDIN (%),0.224467,0.3367,0.224467,9.876543,0.785634,0.785634,27.833895,0.3367,0.3367,0.3367,0.224467,0.785634,0.3367,0.224467,0.224467


In [ ]:
df_traspuesto=describe_df(df).T


In [33]:
df_prueba

,0,1,2,3
COL_N,DATA_TYPE,MISSINGS (%),UNIQUE_VALUES,CARDIN (%)
survived,int64,0.0,2,0.224467
pclass,int64,0.0,3,0.3367
sex,object,0.0,2,0.224467
age,float64,19.86532,88,9.876543
sibsp,int64,0.0,7,0.785634
parch,int64,0.0,7,0.785634
fare,float64,0.0,248,27.833895
embarked,object,0.224467,3,0.3367
class,object,0.0,3,0.3367


#### Función tipifica variables

In [ ]:
#Creamos el df de tipificación
def tipifica_variables(df,umbral_categoria=6, umbral_continua=10):
    Tipo=df.dtypes
    Card=df.nunique()
    Card_rel=Card/len(df)*100
    df_tipificacion=pd.DataFrame([df.columns, Tipo, Card, Card_rel]).T.rename(columns = {0: "nombre_variable", 1: "Tipo", 2: "Card", 3: "Card_rel"})
    df_tipificacion["tipo_sugerido"]="sin categoría" 
    df_tipificacion.loc[df_tipificacion.Card == 2, "tipo_sugerido"] = "Binaria"
    es_numerica = ((df_tipificacion.Tipo == "float64") | (df_tipificacion.Tipo == "int64")) 
    #df_tipificacion.loc[df_tipificacion.Tipo == "datetime64[ns]", "Clasificada_como"] = "Fecha"
    df_tipificacion.loc[(df_tipificacion.Card > 2) & (df_tipificacion.Card < umbral_categoria), "tipo_sugerido"] = "Categórica"
    df_tipificacion.loc[(df_tipificacion.Card > umbral_categoria) & es_numerica & (df_tipificacion.Card_rel < umbral_continua), "tipo_sugerido"] = "Numérica discreta"
    df_tipificacion.loc[(df_tipificacion.Card > umbral_categoria) & es_numerica &  (df_tipificacion.Card_rel > umbral_continua), "tipo_sugerido"] = "Numérica continua"

    return df_tipificacion[["nombre_variable", "tipo_sugerido"]]

In [72]:
tipifica_variables(df)

,nombre_variable,tipo_sugerido
0,survived,Binaria
1,pclass,Categórica
2,sex,Binaria
3,age,Numérica discreta
4,sibsp,Numérica discreta
5,parch,Numérica discreta
6,fare,Numérica continua
7,embarked,Categórica
8,class,Categórica
9,who,Categórica


#### Función get_features_num_regression

In [100]:
from scipy.stats import pearsonr

In [ ]:
#Resumen:
#target_col debe ser una variable numérica continua o discreta con alta cardinalidad
#umbral_corr debe estar entre 0 y 1
#pvalue debe ser por defecto None


def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    df_tip=tipifica_variables(df)
    if target_col in df.columns:
        cond_target1=df_tip.nombre_variable == target_col
        cond_target2=(df_tip.tipo_sugerido == "Numérica continua")
        cond_num= (df_tip.tipo_sugerido == "Numérica continua") | (df_tip.tipo_sugerido == "Numérica discreta")
        target_numerico = df_tip.loc[cond_target1 & cond_target2]
        
        if not target_numerico.empty:
            col_num=df_tip.loc[cond_num, "nombre_variable"].tolist()
            matriz_correlacion=df[col_num].corr()
            correlacion_con_target = matriz_correlacion[target_col].abs()
            print(f"La variable {target_col} se puede considerar variable target, ya que es numérica continua.")
            condicion_umbral=(correlacion_con_target > umbral_corr) & (correlacion_con_target < 1.0)
            num_selec1=correlacion_con_target[condicion_umbral].index.tolist()
            num_selec2 = []
            for col in num_selec1:
            # Eliminamos filas con nulos en estas dos columnas para evitar errores en el test
                df_temp = df[[col, target_col]].dropna()
            # Calculamos la correlación y el p-value:
                r_val, p_val = pearsonr(df_temp[col], df_temp[target_col])
            # Si el usuario no definió pvalue, o si el p_val es menor al umbral solicitado
                if pvalue is None or p_val < pvalue:
                    num_selec2.append(col)
                    print(f"Variable '{col}' seleccionada: p-value = {p_val:.4f} (relación estadísticamente significativa).")
                else:
                    print(f"Variable '{col}' descartada: p-value = {p_val:.4f} (relación no significativa).")
            return num_selec2
        else:
            print(f"La variable {target_col} existe, pero no es numérica continua.")
    else:
        print("La variable no está en el dataset.")




In [117]:
get_features_num_regression(df, "fare", 0.05, 0.005)

La variable fare se puede considerar variable target, ya que es numérica continua.
Variable 'age' descartada: p-value = 0.0102 (relación no significativa).
Variable 'sibsp' seleccionada: p-value = 0.0000 (relación estadísticamente significativa).
Variable 'parch' seleccionada: p-value = 0.0000 (relación estadísticamente significativa).


['sibsp', 'parch']